In [1]:
import glob
import multiprocessing
import os
import sys
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from pandas.core.common import flatten
import uuid
from tqdm import tqdm
from datetime import datetime
import json

import pandas as pd
from pandas.core.frame import DataFrame

OUTPUT_PATH="data/measurements"

In [2]:
files = os.listdir(OUTPUT_PATH)
files

['2021-30-21.parquet',
 '2021-31-336.parquet',
 '2021-32-336.parquet',
 '2021-33-336.parquet',
 '2021-34-336.parquet',
 '2021-35-336.parquet',
 '2021-36-336.parquet',
 '2021-37-336.parquet',
 '2021-38-336.parquet',
 '2021-39-336.parquet',
 '2021-40-9.parquet']

In [3]:
parquetfp = os.path.join(OUTPUT_PATH, files[1])

In [4]:
df = pd.read_parquet(parquetfp)
df.describe()

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
count,336.000000,336.0,336.000000,336.0,336.0,336.000000,336.000000,336.0,336.0,336.000000,...,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.0,336.000000,336.0
mean,0.994048,1.0,1.002976,1.0,1.0,0.976190,1.002976,1.0,1.0,0.994048,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.985119,1.0
std,0.077036,0.0,0.054554,0.0,0.0,0.152683,0.094585,0.0,0.0,0.077036,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.121257,0.0
min,0.000000,1.0,1.000000,1.0,1.0,0.000000,0.000000,1.0,1.0,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,1.0
25%,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
50%,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
75%,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0
max,1.000000,1.0,2.000000,1.0,1.0,1.000000,2.000000,1.0,1.0,1.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.0


In [5]:
df = pd.read_parquet("data/mstatsdf.parquet")
df.describe()

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
count,3054.000000,3054.000000,3054.000000,3054.000000,3054.0,3054.000000,3054.000000,3054.0,3054.000000,3054.000000,...,3054.0,3054.000000,3054.0,3054.000000,3054.000000,3054.000000,3054.000000,3054.000000,3054.000000,3054.0
mean,0.993124,0.999673,0.997708,0.998690,1.0,0.995416,0.992141,1.0,0.997380,0.995088,...,1.0,0.997708,1.0,0.998035,0.998035,0.999673,0.998035,0.998363,0.994434,1.0
std,0.082651,0.018095,0.054246,0.036173,0.0,0.067562,0.111288,0.0,0.051122,0.069922,...,0.0,0.047829,0.0,0.044288,0.044288,0.018095,0.057198,0.040436,0.074413,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.000000,...,1.0,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
50%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
75%,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
max,1.000000,1.000000,2.000000,1.000000,1.0,1.000000,2.000000,1.0,1.000000,1.000000,...,1.0,1.000000,1.0,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.0
